# Initial model: BERT [CLS] token as feature based Tip Mining using TensorFlow
### Model: Bert-base (original)
### Classifiers: LR, XGB and ANN

@author: Erik van der Heide, EUR 2021-2022

### Setup your TensorFlow and Colab Runtime

Pre-modelling steps:
1. Import tensorflow, check version
2. Clone tensorflow official model garden repository from Github
3. Install requirements to use tensorflow/models repository

After doing the above, **restart** run-time.

In [ ]:
"""
!nvidia-smi
print("\n---")

import tensorflow as tf
print("TensorFlow version:", tf.version.VERSION)
print("\n---")

!git clone --depth 1 -b v2.3.0 https://github.com/tensorflow/models.git
print("\n---")

!pip install -Uqr models/official/requirements.txt
"""

'\n!nvidia-smi\nprint("\n---")\n\nimport tensorflow as tf\nprint("TensorFlow version:", tf.version.VERSION)\nprint("\n---")\n\n!git clone --depth 1 -b v2.3.0 https://github.com/tensorflow/models.git\nprint("\n---")\n\n!pip install -Uqr models/official/requirements.txt\n'

### Hyperparameters

In [ ]:
# Hyperparameters
data_type = 'subset5'       # choose 'subsetX', where X=1,2,3,4,5 or 'full'
max_seq_length = 64         # choose fixed maximum sequence length length
batch_size = 32             # to process data sequentially

### Import libraries and data

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import sys
sys.path.append('models')
import time

from official.nlp.data import classifier_data_lib
from official.nlp.bert import tokenization

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss
from sklearn.metrics import plot_confusion_matrix

In [ ]:
print("TF version  :", tf.__version__)
print("Eager mode  :", tf.executing_eagerly())
print("Hub version :", hub.__version__)
print("GPU is      :", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

TF version  : 2.5.0
Eager mode  : True
Hub version : 0.12.0
GPU is      : available


Note: TensorFlow's *eager execution* is an imperative programming environment that evaluates operations immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later.

### Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Data for debugging
if data_type=='subset': path ="drive/MyDrive/Thesis BA&QM 2021/Data/td_clean_subset.csv"
if data_type=='full'  : path ="drive/MyDrive/Thesis BA&QM 2021/Data/td_clean.csv"     

# Data for running
if data_type=='subset1': path ="drive/MyDrive/Thesis BA&QM 2021/Data/td_clean_subset1.csv"
if data_type=='subset2': path ="drive/MyDrive/Thesis BA&QM 2021/Data/td_clean_subset2.csv"
if data_type=='subset3': path ="drive/MyDrive/Thesis BA&QM 2021/Data/td_clean_subset3.csv"
if data_type=='subset4': path ="drive/MyDrive/Thesis BA&QM 2021/Data/td_clean_subset4.csv"
if data_type=='subset5': path ="drive/MyDrive/Thesis BA&QM 2021/Data/td_clean_subset5.csv"

# Read in data
df = pd.read_csv(path, sep="\t", header=0)
print(path)

drive/MyDrive/Thesis BA&QM 2021/Data/td_clean_subset5.csv


In [ ]:
# Choose which data you want to use
df = df[['sentence', 'tip']]
df.reset_index(drop=True, inplace=True)
df['tip'] = df['tip'].astype(int)
print(f"Tip distribution:\n{df['tip'].value_counts()}")
df.dtypes

Tip distribution:
0    3848
1    3848
Name: tip, dtype: int64


sentence    object
tip          int64
dtype: object

Create a train-validation-test split of 65-15-20 .

In [ ]:
if data_type[-1] in ['0','1','2','3','4','5','6','7','8','9']: 
  rand_state = int(data_type[-1]) 
else:
  rand_state = 0
print(f"Random state: {rand_state} \n")

# Train+validation and test split
X_trainval, X_test, y_trainval, y_test = train_test_split(df['sentence'], df['tip'], random_state=rand_state,  train_size=0.8, stratify=df['tip'])

# Train and validation split
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=0, train_size = 0.9998, stratify=y_trainval) # 0.65/0.8 , 0.9998

# Reset indices
#X_train.reset_index(drop=True, inplace=True), y_train.reset_index(drop=True, inplace=True)
#X_val.reset_index(drop=True, inplace=True), y_val.reset_index(drop=True, inplace=True)
#X_test.reset_index(drop=True, inplace=True), y_test.reset_index(drop=True, inplace=True)

# Sizes & example:
print(f"Train size : {X_train.size} - {100*round(X_train.size/df.shape[0], 2)}% - ratio {round(y_train.value_counts(normalize=True)[0],1)}-{round(y_train.value_counts(normalize=True)[1],1)}")
print(f"Valid size : {X_val.size} - {100*round(X_val.size/df.shape[0], 2)}% - ratio {round(y_val.value_counts(normalize=True)[0],1)}-{round(y_val.value_counts(normalize=True)[1],1)}") 
print(f"Test size  : {X_test.size} - {100*round(X_test.size/df.shape[0], 2)}% - ratio {round(y_test.value_counts(normalize=True)[0],1)}-{round(y_test.value_counts(normalize=True)[1],1)}") 
#print(f"\nExample train data: X = '{X_train[0]}'  y = {y_train[0]}")

Random state: 5 

Train size : 6154 - 80.0% - ratio 0.5-0.5
Valid size : 2 - 0.0% - ratio 0.5-0.5
Test size  : 1540 - 20.0% - ratio 0.5-0.5


### Download a pre-trained BERT model for TensorFlow Hub

We load the model we want to use, including the WordPiece vocabulary, lower case command and the tokenizer. 
* BERT-Base: see https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2 for more details about the model.

In [ ]:
label_list = [0,1]  # Label categories [0=non-tip, 1=tip]

# Wrap a saved model in a Keras Layer
URL = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"
bert_layer = hub.KerasLayer(URL, trainable=False) #if False, you freeze params

# Tensor to numpy of vocab, lower case and tokenizer
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

Example of how the WordPiece tokenizer works ([CLS] and [SEP] tokens are still missing here):

In [ ]:
example = "hello, how are you doing today?"
print(f"Example   : {example}")
print(f"Tokens    : {tokenizer.wordpiece_tokenizer.tokenize(example)}")
print(f"Token ids : {tokenizer.convert_tokens_to_ids(tokenizer.wordpiece_tokenizer.tokenize(example))}")

Example   : hello, how are you doing today?
Tokens    : ['hello', '##,', 'how', 'are', 'you', 'doing', 'today', '##?']
Token ids : [7592, 29623, 2129, 2024, 2017, 2725, 2651, 29632]


### Tokenize and preprocess text for BERT

Define a function to convert the input data to inputs for the BERT model:
* input_ids: id's of tokens in sentence (e.g. [101 312 102 0 0])
* input_mask: which tokens the model should focus on (e.g. [1 1 1 0 0])
* segment_ids: only useful if we have input pairs (for us [0 0 0 0 0])

In [ ]:
def to_feature(text, label, label_list=label_list, max_seq_length=max_seq_length, tokenizer=tokenizer):
  example = classifier_data_lib.InputExample(guid=None, # we work per data example
                                             text_a=text.numpy(),
                                             text_b=None, # we do not use paired dataa
                                             label=label.numpy())
  
  feature = classifier_data_lib.convert_single_example(0, example, # 0th (only) example
                                                       label_list, 
                                                       max_seq_length,
                                                       tokenizer)
  
  return (feature.input_ids, feature.input_mask, feature.segment_ids, feature.label_id)

### Wrap a Python function into a TensorFlow op for Eager Execution

We want to use Dataset.map, but that runs in graph mode, and graph tensors do not have a value.

In [ ]:
# Function for Graph Tensors to regular Tensors
# tf.py_function: wraps a python function into a TensorFlow op that executes it eagerly
def to_feature_map(text, label):
  input_ids, input_mask, segment_ids, label_id= tf.py_function(to_feature, 
                                                               inp=[text, label],
                                                               Tout=[tf.int32,tf.int32,tf.int32,tf.int32])
  input_ids.set_shape([max_seq_length])
  input_mask.set_shape([max_seq_length])
  segment_ids.set_shape([max_seq_length])
  label_id.set_shape([])

  x = {
      'input_word_ids': input_ids,
      'input_mask': input_mask,
      'input_type_ids': segment_ids
  }

  return (x, label_id)

### Add a classification head to the BERT layer

Create BERT model with its 3 inputs and a Dense layer with dropout.

In [ ]:
# Building the model
def create_model():
  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                         name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                     name="input_mask")
  input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="input_type_ids")
  
  pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])
  # pooled_output is CLS token, which we want to output
  
  model = tf.keras.Model(
      inputs={
        'input_word_ids': input_word_ids,
        'input_mask': input_mask,
        'input_type_ids': input_type_ids
      },
      outputs=pooled_output)
  
  return model

### Collect [CLS] token

Prepare the data into the right format.

In [ ]:
with tf.device('/cpu:0'): # run on cpu
  train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
  valid_data = tf.data.Dataset.from_tensor_slices((X_val, y_val))
  test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

  # Show the first tensor slice of the training data [take(1) is only the first (text, label) pair]
for text, label in train_data.take(1):
  print(text.numpy())
  print(label.numpy())

b'More caulk ended up on the glass and exterior walls of my house than in the cracks around the windows.'
0


In [ ]:
with tf.device('/cpu:0'):
  # Train
  train_data = (train_data.map(to_feature_map,
                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
  .batch(batch_size,drop_remainder=False)
  .prefetch(tf.data.experimental.AUTOTUNE))

  # Valid
  valid_data = (valid_data.map(to_feature_map,
                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
  .batch(batch_size,drop_remainder=False)
  .prefetch(tf.data.experimental.AUTOTUNE))

  # Test
  test_data = (test_data.map(to_feature_map,
                               num_parallel_calls=tf.data.experimental.AUTOTUNE)
  .batch(batch_size,drop_remainder=False)
  .prefetch(tf.data.experimental.AUTOTUNE))

We feed the training data to the model and save the hidden state of the [CLS] token, as this can be seen as a sentence representation. We only take the training data and not validation data to make a fair comparison with other models.

In [ ]:
# Create the model instance
model = create_model()

# Collect hidden state
start = time.time()
train_cls = model.predict(train_data)
end = time.time()
print("Elapsed time tokenization: ", (end-start)/60, "min\n")

print("Shape   : ", train_cls.shape)
print("Example : ", train_cls[0][0:10],"...")

train_labels = y_train
print("Shape labels : ", train_labels.shape)

Elapsed time tokenization:  0.8312004844347636 min

Shape   :  (6154, 768)
Example :  [-0.83198625 -0.38668388 -0.8685194   0.5719345   0.7176042  -0.18473779
  0.7026555   0.10160547 -0.6444204  -0.99992555] ...
Shape labels :  (6154,)


Do the same for the validation data and test data.

In [ ]:
# Validation data
start = time.time()
valid_cls = model.predict(valid_data)
end = time.time()
print("Elapsed time tokenization: ", (end-start)/60, "min\n")

print("Shape   : ", valid_cls.shape)
print("Example : ", valid_cls[0][0:10],"...")

valid_labels = y_val
print("Shape labels : ", valid_labels.shape)

Elapsed time tokenization:  0.0009561220804850261 min

Shape   :  (2, 768)
Example :  [-0.64129215 -0.3321332  -0.928082    0.69086784  0.8137915  -0.11740386
  0.76416     0.15133017 -0.7383154  -0.9999286 ] ...
Shape labels :  (2,)


In [ ]:
# Test data
start = time.time()
test_cls = model.predict(test_data)
end = time.time()
print("Elapsed time tokenization: ", (end-start)/60, "min\n")

print("Shape   : ", test_cls.shape)
print("Example : ", test_cls[0][0:10],"...")

test_labels = y_test
print("Shape labels : ", test_labels.shape)

Elapsed time tokenization:  0.341607129573822 min

Shape   :  (1540, 768)
Example :  [-0.7388773  -0.28507316 -0.8101706   0.6603096   0.64142144 -0.26226175
  0.45954365  0.2608935  -0.539273   -0.9998473 ] ...
Shape labels :  (1540,)


### Train classifier

Choose your classifier (can specify this after preparing the data)

In [ ]:
classifier_type = 'lr'      # choose classifier 'LR', 'XGB', or 'ANN'

Classifier 1: **Logistic Regression** (LR). \\
No parameter tuning is done. \\
The solver "liblinear" is used to take care of a bug in the LR model. 

In [ ]:
if classifier_type.lower() == 'lr':
  classifier = LogisticRegression(random_state=1, solver='liblinear')

Classifier 2: **eXtreme Gradient Boosting** (XGB). \\
Possibility to tune on validation set, e.g. max. depth $\in [5,10,15,20,25,30]$, but that requires about 1,5 min per parameter value.
Probeer ook sklearn.ensemble.GradientBoostingClassifier.

In [ ]:
if classifier_type.lower() == 'xgb':
  """ # Hyperparameter tuning
  start = time.time()
  best_metric = 0
  best_max_depth = 0
  for depth in range(5, 35, 5):
    classifier_tuning = XGBClassifier(random_state=1, max_depth=depth)
    classifier_tuning.fit(train_cls, train_labels)
    preds_tuning = classifier.predict(valid_cls)
    metric = accuracy_score(valid_labels, preds_tuning)
    if metric > best_metric:
      best_metric = metric
      best_max_depth = depth
  classifier = XGBClassifier(random_state=1, max_depth=best_max_depth)
  end = time.time()
  print("Elapsed time tuning: ", (end-start)/60, "min")
  """
  classifier = XGBClassifier(random_state=1)

Classifier 3: **Artificial Neural Network** (ANN). \\
Parameter tuning is done on the validation set.

In [ ]:
if classifier_type.lower() == 'ann':
  classifier = tf.keras.models.Sequential()
  classifier.add(tf.keras.layers.Dropout(rate=0.1))
  classifier.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
  classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Fit classifier

In [ ]:
train_cls.shape, train_labels.shape

((6154, 768), (6154,))

In [ ]:
print("Classifier: ", classifier_type)
start = time.time()
if classifier_type == 'ann':
  classifier.fit(train_cls, train_labels, batch_size = 32, epochs = 10, validation_data=(valid_cls, valid_labels))
else:
  classifier.fit(train_cls, train_labels)
end = time.time()
print("Elapsed time classification: ", (end-start)/60, "min\n")
if classifier_type == 'ann':
  preds = classifier.predict_classes(test_cls)
else:
  preds = classifier.predict(test_cls)
preds_prob = classifier.predict_proba(test_cls)
if classifier_type != 'ann':
  preds_prob = preds_prob[:,1]
print(preds)
print(preds_prob)

Classifier:  lr
Elapsed time classification:  0.0469019889831543 min

[1 0 1 ... 1 1 0]
[0.59581269 0.30985156 0.90095972 ... 0.78875565 0.54073568 0.05169515]


In [ ]:
# Evaluation metrics
PR   = precision_score(test_labels, preds)
RC   = recall_score(test_labels, preds)
F1   = f1_score(test_labels, preds)
ACC  = accuracy_score(test_labels, preds)
AUC  = roc_auc_score(test_labels, preds_prob)
LOSS = log_loss(test_labels, preds_prob)
print(f"Precision : {PR}")
print(f"Recall    : {RC}")
print(f"F1-score  : {F1}")
print(f"Accuracy  : {ACC}")
print(f"AUC       : {AUC}")
print(f"Loss      : {LOSS}")

print(f"COPY:  {round(PR,4)}, {round(RC,4)}, {round(F1,4)}, {round(ACC,4)}, {round(AUC,4)}, {round(LOSS,4)}")

Precision : 0.7341935483870968
Recall    : 0.7389610389610389
F1-score  : 0.736569579288026
Accuracy  : 0.7357142857142858
AUC       : 0.811310507674144
Loss      : 0.529888147352543
COPY:  0.7342, 0.739, 0.7366, 0.7357, 0.8113, 0.5299


@references: 
* https://www.coursera.org/learn/fine-tune-bert-tensorflow/supplement/zbuz9/resources-on-how-bert-works
* https://www.tensorflow.org/text/tutorials/classify_text_with_bert